### 지도학습

분류 - 이진분류

In [1]:
import pandas as pd

In [2]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD[:2]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


#### 전처리

In [3]:
df_TFD_extract = df_TFD[['Survived','Pclass','Age']]
df_TFD_extract

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0
2,1,3,26.0
3,1,1,35.0
4,0,3,35.0
...,...,...,...
886,0,2,27.0
887,1,1,19.0
888,0,3,NaN
889,1,1,26.0


In [4]:
df_TFD_extractd_preprocess = df_TFD_extract.dropna()
df_TFD_extractd_preprocess

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0
2,1,3,26.0
3,1,1,35.0
4,0,3,35.0
...,...,...,...
885,0,3,39.0
886,0,2,27.0
887,1,1,19.0
889,1,1,26.0


#### Scaling & Encoding 
- split 전에 시행해라

##### Encoding With OneHotEncoding

In [5]:
df_TFD_extractd_preprocess['Pclass'].value_counts()

3    355
1    186
2    173
Name: Pclass, dtype: int64

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
oneHotEncoder = OneHotEncoder()
# oneHotEncoder.fit_transform(df_TFD_extractd_preprocess[['Pclass']]).toarray() 실제값 확인용
encoded_data = oneHotEncoder.fit_transform(df_TFD_extractd_preprocess[['Pclass']]).toarray()
encoded_data.shape

(714, 3)

In [8]:
columns_name = oneHotEncoder.categories_ #dataframe 만들려면 2개 필요해서 만듬

In [18]:
df_encoded_data = pd.DataFrame(data = encoded_data, columns= columns_name)
df_encoded_data[:2]

,1,2,3
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [49]:
df_encoded_data.index, df_encoded_data.shape

(RangeIndex(start=0, stop=714, step=1), (714, 3))

In [ ]:
df_TFD_extractd_preprocess.index, df_TFD_extractd_preprocess

In [10]:
# df_encoded_data = pd.get_dummies(df_TFD_extractd_preprocess['Pclass'], prefix='Pclass')

In [11]:
df_TFD_extractd_preprocess = pd.concat([df_TFD_extractd_preprocess, df_encoded_data], axis=1)
df_TFD_extractd_preprocess[:2]

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
0,0,3,22.0,0,0,1
1,1,1,38.0,1,0,0


In [12]:
df_TFD_extractd_preprocess[['Pclass']][:3]

,Pclass
0,3
1,1
2,3


In [13]:
target = df_TFD_extractd_preprocess['Survived']

In [14]:
df_TFD_extractd_preprocess.columns

Index(['Survived', 'Pclass', 'Age', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

In [16]:
features = df_TFD_extractd_preprocess.drop(columns=['Survived','Pclass'])
features.columns

Index(['Age', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

#### MinMaxScaler
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [20]:
from sklearn.preprocessing import MinMaxScaler

In [26]:
minMaxScaler= MinMaxScaler()
features = minMaxScaler.fit_transform(features)
features.shape

(714, 4)

#### 정형화 단계

In [27]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(features,target, random_state=1)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

#### 모델학습

In [33]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(features_train, target_train)

LogisticRegression()

In [35]:
model.coef_, model.intercept_
#(array([[-1.96570398,  1.01616842,  0.06615765, -1.08240605]]), array([0.58789922]))

(array([[-1.96570398,  1.01616842,  0.06615765, -1.08240605]]),
 array([0.58789922]))

In [37]:
model.predict(features_train[250:255])

array([0, 0, 1, 1, 0], dtype=int64)

In [38]:
model.predict_proba(features_train[250:255])

array([[0.77297066, 0.22702934],
       [0.50062379, 0.49937621],
       [0.38852988, 0.61147012],
       [0.25993248, 0.74006752],
       [0.53760412, 0.46239588]])

#### 평가

In [40]:
target_predict = model.predict(features_train)
target_predict.shape

(535,)

In [41]:
target_train_predict = model.predict(features_train)
target_train_predict.shape  # target_train.shape과 동일

(535,)

#### 평가

In [42]:
from sklearn.metrics import accuracy_score # 정확도

In [43]:
accuracy_score(target_train, target_train_predict) #교내 시험

0.6785046728971963

In [44]:
target_test_predict = model.predict(features_test)
target_test_predict.shape #target_test.shape 동일

(179,)

In [45]:
accuracy_score(target_test, target_test_predict)
#이전값 0.7486033519553073 

0.7262569832402235

In [46]:
from sklearn.metrics import classification_report

In [47]:
print(classification_report(target_train, target_train_predict)) #f1-score 중요 0은 죽음 1은 삶

              precision    recall  f1-score   support

           0       0.70      0.77      0.74       312
           1       0.63      0.55      0.59       223

    accuracy                           0.68       535
   macro avg       0.67      0.66      0.66       535
weighted avg       0.67      0.68      0.67       535



In [48]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.76      0.83      0.79       112
           1       0.66      0.55      0.60        67

    accuracy                           0.73       179
   macro avg       0.71      0.69      0.70       179
weighted avg       0.72      0.73      0.72       179



#### 오차 행렬

In [19]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [20]:
confusion_matrix(target_train, target_train_predict)

array([[246,  66],
       [105, 118]], dtype=int64)

In [21]:
precision_score(target_train, target_train_predict)

0.6413043478260869

In [22]:
recall_score(target_train, target_train_predict)

0.5291479820627802